# ClusDur Task 1.3 Notebook

## Results

![Iperf](results_isolated_net.png)

## Reproducing

- Set up nodes using either node_setup.py if you have access to our project, or by following the manual set up instructions below.
- If you are using a stock image you need to manually install iperf3 on both nodes.
- On one compute node run user@cnode1> iperf3 -s, on the other run user@cnode2> iperf3 -c $CNODE1_IP_OR_HOSTNAME -J > results.json, then store the file.
- Run ./plot_results.py results.json to generate a plot.

## Manual node set up

- Generate a keypair 'my_keypair' from the GUI, and use a stock CC-Centos8-stream image instead of our image. The following section of node_setup.py should be modified to use the keypair and image.

In [ ]:
# outside the ClusDur project replace the image_name and key_name with a stock image and a generated keypair respectively.

head = chi.server.create_server(
    "CD_Head",
    reservation_id=chi.lease.get_node_reservation(head_lease['id']),
    image_name='indyscc_head-node',
    network_name='stbl',
    key_name="Test_ssh"
)

compute = chi.server.create_server(
    "CD_Compute",
    reservation_id=chi.lease.get_node_reservation(compute_lease['id']),
    image_name='indyscc_head-node',
    network_name='stbl',
    key_name="Test_ssh"
)
